# **Installing DEAP Library**

In [47]:
pip install deap

# **Import necessary libraries**

In [48]:
import random
import numpy as np
import pandas as pds
from keras.models import Sequential
from keras.datasets import mnist
from deap import base, creator, tools, algorithms
from keras.utils import np_utils
from keras.optimizers import Adam
from keras.layers import Activation, Dropout, BatchNormalization, Dense
from keras.metrics import categorical_crossentropy
from keras.callbacks import EarlyStopping

# **MNIST class -  functions for data loading, training, fitness calculation and evaluation**

In [49]:
# MNIST class
class MNIST():
    def __init__(self,
                 l1_out=512, 
                 l2_out=512, 
                 l1_drop=0.2, 
                 l2_drop=0.2, 
                 bn1=0,
                 bn2=0,
                 batch_size=100, 
                 epochs=10, 
                 validation_split=0.1):
        self.l1_out = l1_out
        self.l2_out = l2_out
        self.l1_drop = l1_drop
        self.l2_drop = l2_drop
        self.bn1 = bn1
        self.bn2 = bn2
        self.batch_size = batch_size
        self.epochs = epochs
        self.validation_split = validation_split
        self.__x_train, self.__x_test, self.__y_train, self.__y_test = self.mnist_data()
        self.__model = self.mnist_model()
        params = """
        l1_out:\t{0}
        l2_out:\t{1}
        l1_drop:\t{2}
        l2_drop:\t{3}
        bn1:\t{4}
        bn2:\t{5}
        batch_size:\t{6}
        epochs:\t{7}
        validation_split:\t{8}
        """.format(self.l1_out, self.l2_out,
                   self.l1_drop, self.l2_drop,
                   self.bn1, self.bn2,
                   self.batch_size, self.epochs,
                   self.validation_split)
        print(params)
        
    # load mnist data from keras dataset
    def mnist_data(self):
        (X_train, y_train), (X_test, y_test) = mnist.load_data()
        X_train = X_train.reshape(60000, 784)
        X_test = X_test.reshape(10000, 784)

        X_train = X_train.astype('float32')
        X_test = X_test.astype('float32')
        X_train /= 255
        X_test /= 255

        Y_train = np_utils.to_categorical(y_train, 10)
        Y_test = np_utils.to_categorical(y_test, 10)
        return X_train, X_test, Y_train, Y_test
    
    # mnist model
    def mnist_model(self):
        model = Sequential()
        model.add(Dense(self.l1_out, input_shape=(784,)))
        if self.bn1 == 0:
            model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Dropout(self.l1_drop))
        model.add(Dense(self.l2_out))
        if self.bn2 == 0:
            model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Dropout(self.l2_drop))
        model.add(Dense(10))
        model.add(Activation('softmax'))
        model.compile(loss='categorical_crossentropy',
                      optimizer=Adam(),
                      metrics=['accuracy'])

        return model
    
    # fit mnist model
    def mnist_fit(self):
        early_stopping = EarlyStopping(patience=0, verbose=1)
        
        self.__model.fit(self.__x_train, self.__y_train,
                       batch_size=self.batch_size,
                       epochs=self.epochs,
                       verbose=0,
                       validation_split=self.validation_split,
                       callbacks=[early_stopping])
    
    # evaluate mnist model
    def evaluate_mnist(self):
        self.mnist_fit()
        
        evaluation = self.__model.evaluate(self.__x_test, self.__y_test, batch_size=self.batch_size, verbose=0)
        return evaluation


# **Run the model for mnist**

In [50]:
# function to run mnist class
def run_mnist(bounds):
    _mnist = MNIST(l1_out=bounds[0], 
                   l2_out=bounds[1], 
                   l1_drop=bounds[2], 
                   l2_drop=bounds[3], 
                   bn1=bounds[4],
                   bn2=bounds[5],
                   batch_size=bounds[6],
                   epochs=bounds[7], 
                   validation_split=bounds[8])
    eval_mnist = _mnist.evaluate_mnist()
    print(eval_mnist)
    return eval_mnist[0],

# **Genetic Algorithm Implementation**

In [51]:
# creator
creator.create('FitnessMax', base.Fitness, weights = (-1.0,))
creator.create('Individual', list , fitness = creator.FitnessMax)

# defining attributes for individual
toolbox = base.Toolbox()

# layer outputs
toolbox.register("l1_out", random.choice, (64, 128, 256, 512, 1024))
toolbox.register("l2_out", random.choice, (64, 128, 256, 512, 1024))
# dropout late
toolbox.register("l1_drop", random.uniform, 0.0, 0.3)
toolbox.register("l2_drop", random.uniform, 0.0, 0.3)
# batchnormalization
toolbox.register("bn1", random.randint, 0, 1)
toolbox.register("bn2", random.randint, 0, 1)
# training
toolbox.register("batch_size", random.choice, (10, 100, 500))
toolbox.register("epochs", random.choice, (5, 10, 20))
toolbox.register("validation_split", random.uniform, 0.0, 0.3)

# register attributes to individual
toolbox.register('individual', tools.initCycle, creator.Individual, 
                 (toolbox.l1_out, toolbox.l2_out, 
                  toolbox.l1_drop, toolbox.l2_drop,
                  toolbox.bn1, toolbox.bn2, 
                  toolbox.batch_size, toolbox.epochs, toolbox.validation_split), 
                  n = 1)
# individual to population
toolbox.register('population', tools.initRepeat, list, toolbox.individual)

# evolution
toolbox.register('mate', tools.cxTwoPoint)
toolbox.register('mutate', tools.mutFlipBit, indpb = 0.05)
toolbox.register('select', tools.selTournament, tournsize=3)
toolbox.register('evaluate', run_mnist)

/usr/local/lib/python3.6/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


# **Hyper parameter optimization on Genetic Algorithm**

In [52]:
def genAlgorithm(population=5, CXPB=0.5, MUTPB=0.2, NGEN=5):
    random.seed(64)
    pop = toolbox.population(n=population)
    
    print("Start of evolution")

    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit
    print("  Evaluated %i individuals" % len(pop))
    
    for g in range(NGEN):
        print("-- Generation %i --" % g)

        offspring = toolbox.select(pop, len(pop))
        offspring = list(map(toolbox.clone, offspring))

        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                print("mate")
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                print("mutate")
                toolbox.mutate(mutant)
                del mutant.fitness.values

        try:
            invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
            fitnesses = map(toolbox.evaluate, invalid_ind)
            for ind, fit in zip(invalid_ind, fitnesses):
                ind.fitness.values = fit
        except AssertionError:
            pass
                
        print("  Evaluated %i individuals" % len(invalid_ind))

        pop[:] = offspring

        try:
            fits = [ind.fitness.values[0] for ind in pop]

            length = len(pop)
            mean = sum(fits) / length
            sum2 = sum(x*x for x in fits)
            std = abs(sum2 / length - mean**2)**0.5

            print("  Min %s" % min(fits))
            print("  Max %s" % max(fits))
            print("  Avg %s" % mean)
            print("  Std %s" % std)
        except IndexError:
            pass

    print("-- End of (successful) evolution --")

    best_individual = tools.selBest(pop, 1)[0]
    print("Best individual is %s, %s" % (best_individual, best_individual.fitness.values))
    return best_individual

# **Generating the best parameters**

In [53]:
best_individuals = genAlgorithm(population=5, CXPB=0.5, MUTPB=0.2, NGEN=10)

Start of evolution

        l1_out:	512
        l2_out:	64
        l1_drop:	0.18906571629566554
        l2_drop:	0.12118829087054188
        bn1:	0
        bn2:	0
        batch_size:	100
        epochs:	20
        validation_split:	0.27380697683392835
        
Epoch 00003: early stopping
[0.09775235503911972, 0.9706000089645386]

        l1_out:	128
        l2_out:	128
        l1_drop:	0.023943619120116687
        l2_drop:	0.12502013021715647
        bn1:	0
        bn2:	0
        batch_size:	10
        epochs:	10
        validation_split:	0.20383943374115368
        
Epoch 00004: early stopping
[0.08563189208507538, 0.9745000004768372]

        l1_out:	64
        l2_out:	64
        l1_drop:	0.052369065073309795
        l2_drop:	0.17600891153302792
        bn1:	1
        bn2:	0
        batch_size:	500
        epochs:	5
        validation_split:	0.026646503392500687
        
[0.11365970224142075, 0.9646000266075134]

        l1_out:	64
        l2_out:	512
        l1_drop:	0.1191038755140

In [54]:
optbounds = {
    'l1_out' : best_individuals[0],
    'l2_out' : best_individuals[1],
    'l1_drop' : best_individuals[2],
    'l2_drop' : best_individuals[3],
    'bn1' : best_individuals[4],
    'bn2' : best_individuals[5],
    'batch_size' : best_individuals[6],
    'epochs' : best_individuals[7],
    'validation_split' : best_individuals[8]
}

In [55]:
print(optbounds)

{'l1_out': 512, 'l2_out': 64, 'l1_drop': 0.18906571629566554, 'l2_drop': 0.12118829087054188, 'bn1': 0, 'bn2': 0, 'batch_size': 10, 'epochs': 10, 'validation_split': 0.27380697683392835}
